- link nio uids to semantic types
- link nio uids to canonicals 

In [1]:
import urllib.parse
import time
import xmltodict
from xml.dom import minidom
from collections import Counter
import json


In [2]:
# import NIO dictionary token id and canonicals
# the canonicals with be the elements inside the queries to be encoded 
xml_path = "/Users/yidesdo21/Projects/inputs/dictionary/"

with open(xml_path+"nio_ado_case.xml") as f:
    nio_xml = f.read()

nio_parsed = xmltodict.parse(nio_xml)

In [3]:
nio_dict = nio_parsed["synonym"]["token"]

In [4]:
nio_dict[:5]

[OrderedDict([('@id', 'NDDO:Geriatric_Depression_Scale_Assessment_design'),
              ('@canonical', 'Geriatric Depression Scale Assessment design'),
              ('variant',
               OrderedDict([('@base',
                             'Geriatric Depression Scale Assessment design')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000021'),
              ('@canonical', 'visit'),
              ('variant', OrderedDict([('@base', 'visit')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000022'),
              ('@canonical', 'diagnostic data'),
              ('variant', OrderedDict([('@base', 'diagnostic data')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000023'),
              ('@canonical', 'MRI image'),
              ('variant', OrderedDict([('@base', 'MRI image')]))]),
 OrderedDict([('@id', 'NDDO:NDDO_00000024'),
              ('@canonical', 'clinical data'),
              ('variant', OrderedDict([('@base', 'clinical data')]))])]

In [8]:
# nio_canonical_uid = dict()

# for nio in nio_dict:
#     token_id, cano = nio["@id"], nio["@canonical"]
#     nio_canonical_uid[token_id] = cano

In [9]:
# nio_canonical_uid

{'NDDO:Geriatric_Depression_Scale_Assessment_design': 'Geriatric Depression Scale Assessment design',
 'NDDO:NDDO_00000021': 'visit',
 'NDDO:NDDO_00000022': 'diagnostic data',
 'NDDO:NDDO_00000023': 'MRI image',
 'NDDO:NDDO_00000024': 'clinical data',
 'NDDO:NDDO_00000025': 'biomarker data',
 'NDDO:NDDO_00000027': 'PET image',
 'NDDO:NDDO_00000030': 'image data score',
 'NDDO:NDDO_00000036': 'Telephone visit',
 'NDDO:NDDO_00000037': 'Physical visit',
 'NDDO:NDDO_00000040': 'hasAgent',
 'NDDO:NDDO_00000041': 'expressedIn',
 'NDDO:NDDO_00000042': 'extractedFrom',
 'NDDO:NDDO_00000043': 'hasDatatype',
 'NDDO:NDDO_00000044': 'hasID',
 'NDDO:NDDO_00000045': 'hasInformation',
 'NDDO:NDDO_00000046': 'hasInput',
 'NDDO:NDDO_00000047': 'hasOutput',
 'NDDO:NDDO_00000048': 'hasDate',
 'NDDO:NDDO_00000049': 'hasValue',
 'NDDO:NDDO_00000050': 'isOfGender',
 'NDDO:NDDO_00000051': 'isOutputOf',
 'NDDO:NDDO_00000052': 'clinical assessment questionnaire',
 'NDDO:NDDO_00000053': 'isPreceededBy',
 'NDDO:

### breakdown of four areas, ontologies, and uids
- MF -- cognitive functions, CogFunc
    - "obo:MF_...": , 59
    - "obo:GO_...": visual perception, sensory perception, behavior, memory, 66 -> CogFunc
    - "obo:NBO_...": movement, behavior, posture, reflex, 489 -> CogFunc
- NPT -- neuropsychological tests
    - "obo:NPT_...": 470
- ADO -- AD diagnosis, treatment, and molecular mechanisms, AD_DTM
    - "NDDUO:...", 218
    - "AlzheimerOntology:...", 1304
    - "obo:OBI_...": diagnosis, 273 -> AD_DTM
    - "obo:PATO_...": age, quality, handedness, 7, -> AD_DTM
- FMA, NDDO -- brain areas, FMA for brain anatomy and NDDO for brain imaging 
    - "NDDO:...", 840
    - "obo:FMA_...", 619
- under the "AD -- others" groups, i.e., the uids don't belong to any of the above
    - "obo:NCBITaxon...": animals, 13 -> PTC species? Is the hierarchy on the same level?
    - "obo:ND_...": disease names, rare disorder names, 108 -> PTC disease

    - regions, entities, predicates, units, descriptors, data types: 
        - "obo:IAO_...": measure, label, diagram, and predicates, e.g. is duration of, is quality measured as, 113 
        - "obo:RO_...": predicates, e.g., "negatively_regulates", "surrounded by", "has member of", 34 
        - "obo:fma_...": predicates, e.g. attaches to, bounds, 6
        - "oboInOwl:...": predicates, e.g. has exact synonym, 10 
        - "obo:BFO_..." is about regions, entities, and predicates, e.g., precedes, occurs in, 51
        - "bfo:..." is about regions and entities, e.g. temporal region, spacial region, processual_entity, 9     
        - "snap:...": region, continuant, entity, 21 
        - "span:...": region, context, boundary, 17
        - "obo:UO_...": units, e.g. meter, gram, hertz, 55
        - "dc:..." is about descriptors, e.g. date, format, region, title, 15
        - "OntoDT:OntoDT_..." is about data types, 32 
        - "obo:OGMS_...": description of phenotype, genotype, e.g., "clinical phenotype", 11 
        - "obo:UBERON_0000481": "multi-tissue structure", 1 
        - "go:results_in", 1 
        - "iao:IAO_0000599", has id prefix, 1 

- rubbish nio uid that needs to be removed: 
    - "http://www.ifomis.org/bfo/1.1#Entity", 1
    - "rdfs:label", 1 
    - "AlzheimerOntology", 1

In [18]:
token_ids, canos = list(), list()
# onto_list = list()
# obo_list = list()
nio_category_uid = dict()   # {uid:type}
cog_func = ["obo:MF_","obo:GO_","obo:NBO_"]
neuro_test = ["obo:NPT_"]
ad_dtm = ["NDDUO:","AlzheimerOntology:","obo:OBI_","obo:PATO_"]
brain_area = ["NDDO:","obo:FMA_"]
species = ["obo:NCBITaxon"]
disease = ["obo:ND_"]
ad_prep = ["obo:IAO_","obo:RO_","obo:fma_","oboInOwl:","obo:BFO_","bfo:",
           "snap:","span:","obo:UO_","dc:","OntoDT:OntoDT_","obo:OGMS_",
           "obo:UBERON_","go:","iao:IAO_"]

for nio in nio_dict:
    token_id, cano = nio["@id"], nio["@canonical"]
#     check_onto = token_id.split(":")[0]
#     print(check_onto)
#     onto_list.append(check_onto)
#     onto = check_onto
#     print(token_id)
    
    if token_id.startswith(tuple(cog_func)):
        nio_category_uid[token_id] = "CogFunc"
    
    elif token_id.startswith(tuple(neuro_test)):
        nio_category_uid[token_id] = "NeuroTest"    
        
    elif token_id.startswith(tuple(ad_dtm)):
        nio_category_uid[token_id] = "AD-DTM"
        
    elif token_id.startswith(tuple(brain_area)):
        nio_category_uid[token_id] = "BrainArea"    
        
    elif token_id.startswith(tuple(species)):
        nio_category_uid[token_id] = "Species" 
    
    elif token_id.startswith(tuple(disease)):
        nio_category_uid[token_id] = "Disease" 
    
    elif token_id.startswith(tuple(ad_prep)):
        nio_category_uid[token_id] = "ADPrep"  
        
    
#     if check_onto == "obo":
#         obo_check = token_id.split(":")[1].split("_")[0]
#         obo_list.append(obo_check)
#         onto = obo_check
    
#     if onto == "MF":
#         nio_category_uid[token_id] = "AD -- Cognitive function"
#     elif onto == "NPT":
#         nio_category_uid[token_id] = "AD -- Neuropsychological tests"
#     elif onto == "AlzheimerOntology" or onto == "NDDUO":
#         nio_category_uid[token_id] = "AD -- Diagnosis, treatment, and molecular mechanisms"
#     elif onto == "FMA" or onto == "NDDO":
#         nio_category_uid[token_id] = "AD -- Brain areas"
#     else:
#         nio_category_uid[token_id] = "AD -- Others"
        
#         if obo_check == "OBI":
#             pass
#             print(token_id, cano)
#         print(token_id, cano)

In [19]:
len(nio_category_uid)

4842

In [20]:
Counter(nio_category_uid.values())

Counter({'BrainArea': 1459,
         'AD-DTM': 1802,
         'ADPrep': 376,
         'CogFunc': 614,
         'Species': 13,
         'Disease': 108,
         'NeuroTest': 470})

In [21]:
path = "/Users/yidesdo21/Projects/outputs/12_time_slicing/metadata/"

with open(path+'nio_category_uid.json', 'w') as fp:
    json.dump(nio_category_uid, fp, sort_keys=True, indent=4)

In [15]:
Counter(onto_list)

Counter({'NDDO': 840,
         'NDDUO': 217,
         'dc': 15,
         'go': 1,
         'iao': 1,
         'obo': 2376,
         'oboInOwl': 10,
         'rdfs': 1,
         'AlzheimerOntology': 1306,
         'OntoDT': 32,
         'bfo': 9,
         'http': 1,
         'snap': 21,
         'span': 17})

In [29]:
# what's inside obo
Counter(obo_list)

Counter({'BFO': 51,
         'FMA': 619,
         'GO': 66,
         'IAO': 113,
         'IDO': 1,
         'MF': 59,
         'NBO': 489,
         'NCBITaxon': 13,
         'ND': 108,
         'NPT': 470,
         'OBI': 273,
         'OGMS': 11,
         'PATO': 7,
         'RO': 34,
         'UBERON': 1,
         'UO': 55,
         'fma': 6})

MF -- cognitive functions
NPT -- neuropsychological tests
ADO, NDDUO -- AD diagnosis, treatment, and molecular mechanisms
FMA, NDDO -- brain areas, FMA for brain anatomy and NDDO for brain imaging 

- NDDO (Neurodegenerative Disease Data Ontology, Kostovska et al., 2019) -- for brain imaging on AD and Parkinson’s disease.
- NDDUO -- the upper ontology for ADO, judging from the uids, it's about the contexts, rates, e.g. "outcome context", "specific mortality rate", not really about AD
- obo  
    -- BFO, the upper ontology of ND (Jensen et al., 2013), NPT (Cox et al., 2013), ADO (Malhotra et al., 2014); from the NIO uids, the bfo uids are about the prepositions
    -- FMA (Foundation Model of Anatomy, Rosse and Mejino, 2008), anatomy of human body, with priority on the brain structure
    -- GO
    -- IAO
    -- MF (Mental Functioning Ontology, Hastings et al., 2012), cognitive function, mental function
    -- NBO
    -- NCBITaxon
    -- ND, (Jensen et al., 2013), The Neurological Disease Ontology, "a set of controlled classes that describe factors, such as range, signs and symptoms of neurodegenerative diseases, and evaluations, diagnoses, and medical interventions that have been found in the course of clinical practice" (Gomez-Valades, 2021).
    -- NPT, (Cox et al., 2013), Neuro-Psychological Testing, focusing on AD and other neurodegenerative diseases and their early detection
    -- OBI
    -- OGMS
    -- PATO
    -- RO
    -- UO
    -- fma
- AlzheimerOntology (Malhotra et al., 2014) -- AD diagnosis, treatment, and molecular mechanisms 